<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png"  width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction

Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

<a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01" target="_blank">Spacex DataSet</a>


### Store the dataset in database table

**it is highly recommended to manually load the table using the database console LOAD tool in DB2**.

<img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload.png">

Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the  dataset, Next create a New Table, and then follow the steps on-screen instructions to load the data. Name the new table as follows:

**SPACEXDATASET**

**Follow these steps while using old DB2 UI which is having Open Console Screen**

**Note:While loading Spacex dataset, ensure that detect datatypes is disabled. Later click on the pencil icon(edit option).**

1.  Change the Date Format by manually typing DD-MM-YYYY and timestamp format as DD-MM-YYYY HH\:MM:SS.

    Here you should place the cursor at Date field and manually type as DD-MM-YYYY.

2.  Change the PAYLOAD_MASS\_\_KG\_  datatype  to INTEGER.

<img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload2.png">


**Changes to be considered when having DB2 instance with the new UI having Go to UI screen**

*   Refer to this insruction in this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sign%20up%20for%20IBM%20Cloud%20-%20Create%20Db2%20service%20instance%20-%20Get%20started%20with%20the%20Db2%20console/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">link</a> for viewing  the new  Go to UI screen.

*   Later click on **Data link(below SQL)**  in the Go to UI screen  and click on **Load Data** tab.

*   Later browse for the downloaded spacex file.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/browsefile.png" width="800"/>

*   Once done select the schema andload the file.

 <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload3.png" width="800"/>


In [1]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa
!pip install ipython-sql

! pip install  ibm_db
import ibm_db
import ibm_db_dbi
import pandas as pd

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
%load_ext sql

**DB2 magic in case of old UI service credentials.**

In the next cell enter your db2 connection string. Recall you created Service Credentials for your Db2 instance before. From the **uri** field of your Db2 service credentials copy everything after db2:// (except the double quote at the end) and paste it in the cell below after ibm_db_sa://

<img src ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_edX/images/URI.jpg">

in the following format

**%sql ibm_db_sa://my-username:my-password\@my-hostname:my-port/my-db-name**

**DB2 magic in case of new UI service credentials.**

<img src ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/servicecredentials.png" width=600>  

*   Use the following format.

*   Add security=SSL at the end

**%sql ibm_db_sa://my-username:my-password\@my-hostname:my-port/my-db-name?security=SSL**


In [ ]:
%sql ibm_db_sa://ypx40671:8vEjnPY1ofxogZvk@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?authSource=admin&replicaSet=replset?security=SSL

In [3]:
def get_db_connection():    
    dsn_driver = "{IBM DB2 ODBC DRIVER}"
    dsn_database ="BLUDB"
    dsn_hostname ="54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
    dsn_port= "32733"
    dsn_protocol = "TCPIP"
    dsn_uid = "ypx40671"
    dsn_pwd =  "8vEjnPY1ofxogZvk"
    dsn_security = "SSL" 
    dsn = (
        "DRIVER={0};"
        "DATABASE={1};"
        "HOSTNAME={2};"
        "PORT={3};"
        "PROTOCOL={4};"
        "UID={5};"
        "PWD={6};"
        "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security  )
    try:
        conn = ibm_db.connect(dsn, "", "")
        print("Connected!")
        return conn
    except Exception:
        print("\nERROR: Unable to connect to the \'" + dsn_database + "\' server.")
        print("error: ", ibm_db.conn_errormsg())
        exit(-1)

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

### Task 1

##### Display the names of the unique launch sites  in the space mission


In [4]:
pconn=ibm_db_dbi.Connection(get_db_connection())
select_query = 'select * from spacextbl'
df=pd.read_sql(select_query,pconn)
df

Connected!


,DATE,TIME__UTC_,BOOSTER_VERSION,LAUNCH_SITE,PAYLOAD,PAYLOAD_MASS__KG_,ORBIT,CUSTOMER,MISSION_OUTCOME,LANDING__OUTCOME
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt
...,...,...,...,...,...,...,...,...,...,...
96,2020-11-05,23:24:23,F9 B5B1062.1,CCAFS SLC-40,"GPS III-04 , Crew-1",4311,MEO,USSF,Success,Success
97,2020-11-16,00:27:00,F9 B5B1061.1,KSC LC-39A,"Crew-1, Sentinel-6 Michael Freilich",12500,LEO (ISS),NASA (CCP),Success,Success
98,2020-11-21,17:17:08,F9 B5B1063.1,VAFB SLC-4E,"Sentinel-6 Michael Freilich, Starlink 15 v1.0",1192,LEO,NASA / NOAA / ESA / EUMETSAT,Success,Success
99,2020-11-25,02:13:00,F9 B5 B1049.7,CCAFS SLC-40,"Starlink 15 v1.0, SpaceX CRS-21",15600,LEO,SpaceX,Success,Success


In [5]:
conn = get_db_connection()
pconn=ibm_db_dbi.Connection(conn)
select_query = 'select distinct LAUNCH_SITE from SPACEXTBL'
df=pd.read_sql(select_query,pconn)
ibm_db.close(conn)
df

Connected!


,LAUNCH_SITE
0,CCAFS LC-40
1,CCAFS SLC-40
2,KSC LC-39A
3,VAFB SLC-4E


### Task 2

##### Display 5 records where launch sites begin with the string 'CCA'


In [9]:
conn = get_db_connection()
pconn=ibm_db_dbi.Connection(conn)
select_query = "select distinct LAUNCH_SITE from SPACEXTBL where LAUNCH_SITE like 'CCA%'"
df=pd.read_sql(select_query,pconn)
ibm_db.close(conn)
df

Connected!


,LAUNCH_SITE
0,CCAFS LC-40
1,CCAFS SLC-40


### Task 3

##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [10]:
conn = get_db_connection()
pconn=ibm_db_dbi.Connection(conn)
select_query = "select sum(PAYLOAD_MASS__KG_) from SPACEXTBL where CUSTOMER = 'NASA (CRS)'"
df=pd.read_sql(select_query,pconn)
ibm_db.close(conn)
df

Connected!


,1
0,45596


### Task 4

##### Display average payload mass carried by booster version F9 v1.1


In [11]:
conn = get_db_connection()
pconn=ibm_db_dbi.Connection(conn)
select_query = "select avg(PAYLOAD_MASS__KG_) from SPACEXTBL where BOOSTER_VERSION like 'F9 v1.1%'"
df=pd.read_sql(select_query,pconn)
ibm_db.close(conn)
df

Connected!


,1
0,2534


### Task 5

##### List the date when the first successful landing outcome in ground pad was acheived.

*Hint:Use min function*


In [16]:
conn = get_db_connection()
pconn=ibm_db_dbi.Connection(conn)
select_query = "select date from SPACEXTBL where LANDING__OUTCOME =  'Success (ground pad)' \
                order by date  limit 1"
df=pd.read_sql(select_query,pconn)
ibm_db.close(conn)
df

Connected!


,DATE
0,2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [21]:

conn = get_db_connection()
pconn=ibm_db_dbi.Connection(conn)
select_query = "select BOOSTER_VERSION from SPACEXTBL where PAYLOAD_MASS__KG_ > 4000 and PAYLOAD_MASS__KG_ <= 6000 \
               and LANDING__OUTCOME = 'Success (drone ship)'"
df=pd.read_sql(select_query,pconn)
ibm_db.close(conn)
df

Connected!


,BOOSTER_VERSION
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


### Task 7

##### List the total number of successful and failure mission outcomes


In [22]:
conn = get_db_connection()
pconn=ibm_db_dbi.Connection(conn)
select_query = "select MISSION_OUTCOME, count(MISSION_OUTCOME) from SPACEXTBL group by MISSION_OUTCOME "
df=pd.read_sql(select_query,pconn)
ibm_db.close(conn)
df

Connected!


,MISSION_OUTCOME,2
0,Failure (in flight),1
1,Success,99
2,Success (payload status unclear),1


### Task 8

##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [24]:
conn = get_db_connection()
pconn=ibm_db_dbi.Connection(conn)
select_query = "select BOOSTER_VERSION from SPACEXTBL where PAYLOAD_MASS__KG_=\
                (select max(PAYLOAD_MASS__KG_) from SPACEXTBL)"
df=pd.read_sql(select_query,pconn)
ibm_db.close(conn)
df

Connected!


,DATE,TIME__UTC_,BOOSTER_VERSION,LAUNCH_SITE,PAYLOAD,PAYLOAD_MASS__KG_,ORBIT,CUSTOMER,MISSION_OUTCOME,LANDING__OUTCOME
0,2019-11-11,14:56:00,F9 B5 B1048.4,CCAFS SLC-40,"Starlink 1 v1.0, SpaceX CRS-19",15600,LEO,SpaceX,Success,Success
1,2020-01-07,02:33:00,F9 B5 B1049.4,CCAFS SLC-40,"Starlink 2 v1.0, Crew Dragon in-flight abort t...",15600,LEO,SpaceX,Success,Success
2,2020-01-29,14:07:00,F9 B5 B1051.3,CCAFS SLC-40,"Starlink 3 v1.0, Starlink 4 v1.0",15600,LEO,SpaceX,Success,Success
3,2020-02-17,15:05:00,F9 B5 B1056.4,CCAFS SLC-40,"Starlink 4 v1.0, SpaceX CRS-20",15600,LEO,SpaceX,Success,Failure
4,2020-03-18,12:16:00,F9 B5 B1048.5,KSC LC-39A,"Starlink 5 v1.0, Starlink 6 v1.0",15600,LEO,SpaceX,Success,Failure
5,2020-04-22,19:30:00,F9 B5 B1051.4,KSC LC-39A,"Starlink 6 v1.0, Crew Dragon Demo-2",15600,LEO,SpaceX,Success,Success
6,2020-06-04,01:25:00,F9 B5 B1049.5,CCAFS SLC-40,"Starlink 7 v1.0, Starlink 8 v1.0",15600,LEO,"SpaceX, Planet Labs",Success,Success
7,2020-09-03,12:46:14,F9 B5 B1060.2,KSC LC-39A,"Starlink 11 v1.0, Starlink 12 v1.0",15600,LEO,SpaceX,Success,Success
8,2020-10-06,11:29:34,F9 B5 B1058.3,KSC LC-39A,"Starlink 12 v1.0, Starlink 13 v1.0",15600,LEO,SpaceX,Success,Success
9,2020-10-18,12:25:57,F9 B5 B1051.6,KSC LC-39A,"Starlink 13 v1.0, Starlink 14 v1.0",15600,LEO,SpaceX,Success,Success


### Task 9

##### List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015


In [28]:
conn = get_db_connection()
pconn=ibm_db_dbi.Connection(conn)
select_query = "select date, BOOSTER_VERSION, LAUNCH_SITE from SPACEXTBL where LANDING__OUTCOME = 'Failure (drone ship)'\
                and year(date) = 2015"
df=pd.read_sql(select_query,pconn)
ibm_db.close(conn)
df

Connected!


,DATE,BOOSTER_VERSION,LAUNCH_SITE
0,2015-01-10,F9 v1.1 B1012,CCAFS LC-40
1,2015-04-14,F9 v1.1 B1015,CCAFS LC-40


### Task 10

##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order


In [31]:
conn = get_db_connection()
pconn=ibm_db_dbi.Connection(conn)
select_query = "select * from(select LANDING__OUTCOME, count(LANDING__OUTCOME) cnt from SPACEXTBL where LANDING__OUTCOME in ('Failure (drone ship)','Success (ground pad)')\
                and date between  '2010-06-04' and '2017-03-20'\
                 group by LANDING__OUTCOME) order by cnt desc"
df=pd.read_sql(select_query,pconn)
ibm_db.close(conn)
df

Connected!


,LANDING__OUTCOME,CNT
0,Failure (drone ship),5
1,Success (ground pad),3


### Reference Links

*   <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>

*   <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*   <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*   <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Hands-on Lab: Analyzing a real World Data Set</a>


## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


## Change log

| Date       | Version | Changed by    | Change Description        |
| ---------- | ------- | ------------- | ------------------------- |
| 2021-10-12 | 0.4     | Lakshmi Holla | Changed markdown          |
| 2021-08-24 | 0.3     | Lakshmi Holla | Added library update      |
| 2021-07-09 | 0.2     | Lakshmi Holla | Changes made in magic sql |
| 2021-05-20 | 0.1     | Lakshmi Holla | Created Initial Version   |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
